In [11]:
from typing import Dict
from string import Template
import pyTigerGraph as tg

schema = {
    'GraphName': 'VWG',
    'VertexTypes': [
        {
            'Config': {'STATS': 'OUTDEGREE_BY_EDGETYPE'},
            'Attributes': [
                {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'LoadCapacity'},
                {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'UnloadCapacity'},
                {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'Stock'},
                {'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'Carga'},
                {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'Capacity'},
                {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'latitude'},
                {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'longitude'}
            ],
            'PrimaryId': {'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'id'},
            'Name': 'Nodes'
        }
    ],
    'EdgeTypes': [
        {
            'IsDirected': False,
            'ToVertexTypeName': 'Nodes',
            'Config': {},
            'Attributes': [
                {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'Price'},
                {'AttributeType': {'Name': 'STRING'}, 'AttributeName': 'Carga'},
                {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'Capacity'},
                {'AttributeType': {'Name': 'FLOAT'}, 'AttributeName': 'Daily_movement'}
            ],
            'FromVertexTypeName': 'Nodes',
            'Name': 'distribute_to'
        }
    ],
    'UDTs': []
}

data_source_dict = {
    "vertex_data_job": r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\data\Europe\biggest_europe_cities_without_CODE.csv",  # Chemin vers le fichier de données pour les sommets
    "edge_data_job": r"C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\data\Europe\conexiones_biggest_europe_cities_without_code_dailymovement80%.csv"  # Chemin vers le fichier de données pour les arêtes
}

create_loading_jobs_template_vwg = Template("""
USE GRAPH $graph_name

CREATE LOADING JOB vertex_data_job FOR GRAPH $graph_name {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO VERTEX Nodes VALUES($$0, $$1, $$2, $$3, $$4, $$5, $$6, $$7) USING SEPARATOR=",", HEADER="true", EOL="\\n";
}
set exit_on_error = "true"
set exit_on_error = "false"

CREATE LOADING JOB edge_data_job FOR GRAPH $graph_name {
    DEFINE FILENAME MyDataSource;
    LOAD MyDataSource TO EDGE distribute_to VALUES($$0, $$1, $$2, $$3, $$4 , $$5) USING SEPARATOR=",", HEADER="true", EOL="\\n";
}
set exit_on_error = "true"
set exit_on_error = "false"
""")


class GraphGenerator:
    def __init__(self, conn, schema: Dict, data_source_dict: Dict):
        # Initialize with connection object, schema, and data sources for vertices and edges.


#         Example of data_source_dict : 
#         DEFAULT_DATA_SOURCE_DICT = {
#                        "vertex_data_job": r"Tigergraph\data\Europe\biggest_europe_citie.csv",
#                        "edge_data_job": r"Tigergraph\data\Europe\conexiones_biggest_europe_cities.csv"
#                        }
#       with vertex_data and edge_data being the name of the loading job

        self.conn = conn
        self.schema = schema
        self.data_source_dict = data_source_dict
        self.graph_name = schema['GraphName']

    def create_loading_jobs(self):
        # Create loading jobs using the template, substituting the graph name.
        try:
            loading_jobs = create_loading_jobs_template_vwg.substitute({
                'graph_name': self.graph_name,
            })
            print(self.conn.gsql(loading_jobs))
        except Exception as e:
            print(f"Error creating loading jobs: {str(e)}")

    def load_from_files(self):
        # Execute loading jobs using the specified data sources.
        created_jobs = self.conn.gsql(f"USE GRAPH {self.graph_name} SHOW JOB *")
        jobs_to_rerun = []

        # Load data from files for each job
        for job_name, file_path in self.data_source_dict.items():
            
            if f"{job_name}" in created_jobs:
                try:
                    print(f"---- Uploading file {file_path} for job {job_name} ---- \n")
                    response = self.conn.runLoadingJobWithFile(
                        filePath=file_path,
                        fileTag="MyDataSource",
                        jobName=f"{job_name}",
                        timeout=32000
                    )
                    print(response)
                except Exception as e:
                    print(f"Job {job_name} failed: {str(e)}")
                    jobs_to_rerun.append(job_name)
            else:
                print(f"Job {job_name} does not exist.")
        
        print(f"Jobs that failed: {jobs_to_rerun}")
        
    def execute_pipeline(self):
        # Execute the full pipeline: loading job creation, and data loading.
        self.create_loading_jobs()
        self.load_from_files()

# Utilisation de la classe
hostName = "https://2cc2f8bde8df444bb60c6fb83491bb8c.i.tgcloud.io"
graphName = "VWG"
secret = "m2p8nba0uab7dtthbn4o1b30r8tqgg9a"
userName = "user_1"
password = "A1z2e3r4*"


conn = tg.TigerGraphConnection(host=hostName, graphname=graphName, username=userName, password=password)
conn.getToken(secret)
graph_generator = GraphGenerator(conn, schema, data_source_dict)
# graph_generator.execute_pipeline()




In [9]:
graph_generator.create_loading_jobs()

Using graph 'VWG'
Successfully created loading jobs: [vertex_data_job].
Successfully created loading jobs: [edge_data_job].


In [12]:
graph_generator.load_from_files()


---- Uploading file C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\data\Europe\biggest_europe_cities_without_CODE.csv for job vertex_data_job ---- 

[{'sourceFileName': 'Online_POST', 'statistics': {'sourceFileName': 'Online_POST', 'parsingStatistics': {'fileLevel': {'validLine': 137}, 'objectLevel': {'vertex': [{'typeName': 'Nodes', 'validObject': 136, 'invalidAttribute': 1}], 'edge': [], 'deleteVertex': [], 'deleteEdge': []}}}}]
---- Uploading file C:\Users\JulienRigot\OneDrive - LIS Data Solutions\Escritorio\code_GORDIAS\base de datos graph\Tigergraph\data\Europe\conexiones_biggest_europe_cities_without_code_dailymovement80%.csv for job edge_data_job ---- 

[{'sourceFileName': 'Online_POST', 'statistics': {'sourceFileName': 'Online_POST', 'parsingStatistics': {'fileLevel': {'validLine': 122}, 'objectLevel': {'vertex': [], 'edge': [{'typeName': 'distribute_to', 'validObject': 121, 'invalidAttribute': 1}], 'deleteVertex': [], 

In [2]:
print(conn.gsql(f"USE GRAPH VWG SHOW JOB *"))


Using graph 'VWG'
  - CREATE LOADING JOB load_nodes FOR GRAPH VWG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Nodes VALUES($0, $1, $2, $3, $4, $5, $6, $7) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

  - CREATE LOADING JOB load_nodes_test FOR GRAPH VWG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Nodes VALUES($0, $1, $2, $3, $4, $5, $6, $7) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

  - CREATE LOADING JOB load_distribute_to FOR GRAPH VWG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO EDGE distribute_to VALUES($0, $1, $2, $3, $4, $5) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

  - CREATE LOADING JOB load_vertex FOR GRAPH VWG {
      DEFINE FILENAME MyDataSource;
      LOAD MyDataSource TO VERTEX Nodes VALUES($0, $1, $2, $3, $4, $5, $6, $7) USING SEPARATOR=",", HEADER="true", EOL="\n";
    }

  - CREATE LOADING JOB load_edge FOR GRAPH VWG {
      DEFINE FILENAME MyDataSource;
      LOAD My